## Eksportowanie modelu



In [30]:
import os
import re
from glob import glob
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.utils.data_utils import get_file
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from tqdm import tqdm
from tensorflow.keras.models import model_from_json
import json
from keras.models import Model
from tensorflow import keras
import time

In [31]:
#model_name = "my_ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8"
model_name = "my_ssd_resnet50_v1_fpn"
#model_name = "my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet152_v1_fpn_640x640_coco17_tpu-8"
#model_name = "my_ssd_resnet101_v1_fpn_640x640_coco17_tpu-8"
keras_model_name = f"{model_name}_keras"

images_path = f"TensorFlow/workspace/training_demo/images"
model_path = f"TensorFlow/workspace/training_demo/models/{model_name}"
exported_model_path = f"TensorFlow/workspace/training_demo/exported-models/"

keras_model_path = f"TensorFlow/workspace/training_demo/models/{keras_model_name}"
weights_path = f"{keras_model_path}/weights"
npy_weights_path = f"{keras_model_path}/npy_weights"

weights_filename = f"{keras_model_name}_weights.h5"
model_filename = f"{keras_model_name}.h5"

ckpt_dict = {"my_ssd_resnet50_v1_fpn":'/ckpt-31',
                    "my_ssd_resnet101_v1_fpn_640x640_coco17_tpu-8":"/ckpt-28",
                    "my_ssd_resnet152_v1_fpn_640x640_coco17_tpu-8":"/ckpt-26",
                    "my_ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8":"/ckpt-26",
                    "my_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8": "/ckpt-51"}



### Eksportowanie wytrenowanego modelu do formatu Tensorflow
za pomocą tf.saved_model.save, freezing graph

In [32]:
export_model_command = f"python TensorFlow/workspace/training_demo/exporter_main_v2.py --input_type image_tensor --pipeline_config_path {model_path}/pipeline.config --trained_checkpoint_dir {model_path}/ --output_directory {exported_model_path}/{model_name}_exported"
print(export_model_command)

python TensorFlow/workspace/training_demo/exporter_main_v2.py --input_type image_tensor --pipeline_config_path TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn/ --output_directory TensorFlow/workspace/training_demo/exported-models//my_ssd_resnet50_v1_fpn_exported


In [33]:
#!{export_model_command}

### Eksportowanie wytrenowanego modelu do formatu Keras


#### Extract layer weights from TF checkpoint

In [43]:
# regex for renaming the tensors to their corresponding Keras counterpart
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')


def get_filename(key):
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = filename.replace(model_name + "_", '')

    # remove "Repeat" scope from filename
    filename = re_repeat.sub('B', filename)

    if re_block8.match(filename):
        # the last block8 has different name with the previous 9 occurrences
        filename = filename.replace('Block8', 'Block8_10')
    elif filename.startswith('Logits'):
        # remove duplicate "Logits" scope
        filename = filename.replace('Logits_', '', 1)

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'

if not os.path.exists(npy_weights_path):
    os.makedirs(npy_weights_path)

reader = tf.train.load_checkpoint(exported_model_path + "/" + model_name + "_exported/checkpoint/")

for key in reader.get_variable_to_shape_map():
    # not saving the following tensors
    if key == 'global_step':
        continue
    if 'AuxLogit' in key:
        continue

    # convert tensor name into the corresponding Keras layer weight name and save 
    arr = reader.get_tensor(key)
    tensor_path = npy_weights_path + "/" + get_filename(key)
    np.save(tensor_path, arr)
    print("tensor_name: ", key)
    #print("tensor_filename: ", get_filename(key))


tensor_name:  save_counter/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-99/beta/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-96/kernel/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/moving_variance/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/moving_mean/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/gamma/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-95/beta/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-93/moving_mean/.ATTRIBUTES/VARIABLE_VALUE
tensor_name:  model/_feature_extractor/classification_backbone/layer_with_weights-93/beta/.ATTRIBUT

#### Load weights

In [41]:
print('Instantiating an empty ResNet50V2 model...')
model = keras.applications.ResNet50V2(
    weights=None,
    input_shape= (640, 640, 3), # Rozmiar modelu 640x640
    classes = 1,
    classifier_activation= 'softmax',
)
model.summary()

Instantiating an empty ResNet50V2 model...
Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 640, 640, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 646, 646, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 320, 320, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 322, 322, 64)         0         ['conv1_conv[0][0]']          
                                              

#### Load NumPy weight files and save to a Keras HDF5 weights file

In [42]:
print('Loading numpy weights from', npy_weights_path)
weights_names = []
for layer in model.layers:
    if layer.weights:
        weights = []
        
        for w in layer.weights:
            weight_name = os.path.basename(w.name).replace(':0', '')
            weight_file = layer.name + '_' + weight_name + '.npy'
            print(f"{layer.name} || {weight_name} || {weight_file}")
            weight_arr = np.load(os.path.join(npy_weights_path, weight_file))
            weights_names.append(weight_file)
            weights.append(weight_arr)
        layer.set_weights(weights)

print('Saving weights...')
model.save_weights(os.path.join(weights_path, weights_filename))
print('Saving model...')
model.save(os.path.join(model_path, model_filename))

Loading numpy weights from TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn_keras/npy_weights
conv1_conv || kernel || conv1_conv_kernel.npy


FileNotFoundError: [Errno 2] No such file or directory: 'TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn_keras/npy_weights/conv1_conv_kernel.npy'